In [55]:
import numpy as np
import pandas as pd

import urllib.request
from PIL import Image

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [56]:
df = pd.read_csv("diamonds_train.csv", index_col=0)
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.21,Ideal,H,VS2,63.0,57.0,6.73,6.70,4.23,6134
1,0.28,Very Good,D,VVS2,64.0,56.0,4.14,4.17,2.66,532
2,0.42,Premium,F,VS1,61.2,58.0,4.86,4.82,2.96,1103
3,0.26,Ideal,H,IF,61.1,57.0,4.16,4.12,2.53,600
4,1.10,Good,G,SI1,63.4,57.0,6.52,6.55,4.14,4997
...,...,...,...,...,...,...,...,...,...,...
40340,1.55,Premium,H,VS2,61.3,61.0,7.46,7.39,4.55,11708
40341,0.36,Ideal,D,SI1,60.6,56.0,4.58,4.63,2.79,619
40342,0.57,Very Good,I,VS2,62.2,55.0,5.33,5.34,3.32,1267
40343,1.01,Very Good,F,IF,59.6,62.0,6.47,6.56,3.88,9965


In [57]:
df.corr()

,carat,depth,table,x,y,z,price
carat,1.000000,0.029414,0.181273,0.975077,0.944445,0.967289,0.921887
depth,0.029414,1.000000,-0.288987,-0.023389,-0.027880,0.097399,-0.009228
table,0.181273,-0.288987,1.000000,0.193986,0.180449,0.153175,0.127338
x,0.975077,-0.023389,0.193986,1.000000,0.966827,0.985080,0.883163
y,0.944445,-0.027880,0.180449,0.966827,1.000000,0.959957,0.857564
z,0.967289,0.097399,0.153175,0.985080,0.959957,1.000000,0.872855
price,0.921887,-0.009228,0.127338,0.883163,0.857564,0.872855,1.000000


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40345 entries, 0 to 40344
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    40345 non-null  float64
 1   cut      40345 non-null  object 
 2   color    40345 non-null  object 
 3   clarity  40345 non-null  object 
 4   depth    40345 non-null  float64
 5   table    40345 non-null  float64
 6   x        40345 non-null  float64
 7   y        40345 non-null  float64
 8   z        40345 non-null  float64
 9   price    40345 non-null  int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 3.4+ MB


In [59]:
x = df.iloc[:,:-1]
x

,carat,cut,color,clarity,depth,table,x,y,z
0,1.21,Ideal,H,VS2,63.0,57.0,6.73,6.70,4.23
1,0.28,Very Good,D,VVS2,64.0,56.0,4.14,4.17,2.66
2,0.42,Premium,F,VS1,61.2,58.0,4.86,4.82,2.96
3,0.26,Ideal,H,IF,61.1,57.0,4.16,4.12,2.53
4,1.10,Good,G,SI1,63.4,57.0,6.52,6.55,4.14
...,...,...,...,...,...,...,...,...,...
40340,1.55,Premium,H,VS2,61.3,61.0,7.46,7.39,4.55
40341,0.36,Ideal,D,SI1,60.6,56.0,4.58,4.63,2.79
40342,0.57,Very Good,I,VS2,62.2,55.0,5.33,5.34,3.32
40343,1.01,Very Good,F,IF,59.6,62.0,6.47,6.56,3.88


In [60]:
y = df['price']
y

0         6134
1          532
2         1103
3          600
4         4997
         ...  
40340    11708
40341      619
40342     1267
40343     9965
40344     1340
Name: price, Length: 40345, dtype: int64

In [61]:
x =  x.drop(['table'], axis=1)
x

,carat,cut,color,clarity,depth,x,y,z
0,1.21,Ideal,H,VS2,63.0,6.73,6.70,4.23
1,0.28,Very Good,D,VVS2,64.0,4.14,4.17,2.66
2,0.42,Premium,F,VS1,61.2,4.86,4.82,2.96
3,0.26,Ideal,H,IF,61.1,4.16,4.12,2.53
4,1.10,Good,G,SI1,63.4,6.52,6.55,4.14
...,...,...,...,...,...,...,...,...
40340,1.55,Premium,H,VS2,61.3,7.46,7.39,4.55
40341,0.36,Ideal,D,SI1,60.6,4.58,4.63,2.79
40342,0.57,Very Good,I,VS2,62.2,5.33,5.34,3.32
40343,1.01,Very Good,F,IF,59.6,6.47,6.56,3.88


In [62]:
for i, val in enumerate(x['depth']):
    if val >= 59.5 and val <= 62.9:
        x['depth'][i] = 4
    elif val > 56.2 and val < 59.5:
        x['depth'][i] = 3
    elif val > 62.9 and val < 65:
        x['depth'][i] = 3
    elif val >= 53.2 and val <= 56.2:
        x['depth'][i] = 2
    elif val >= 65 and val <= 68:
        x['depth'][i] = 2  





ipykernel_launcher:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [63]:
for i,val in enumerate(x['cut']):
    if val == 'Fair':
        x['cut'][i] = 50
    elif val == 'Good':
        x['cut'][i] = 100
    elif val == 'Very Good':
       x['cut'][i] = 150
    elif val == 'Ideal':
       x['cut'][i] = 200
    elif val == 'Premium':
      x['cut'][i] = 250

ipykernel_launcher:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [64]:
df['color'].value_counts()

G    8499
E    7351
F    7130
H    6234
D    5074
I    4023
J    2034
Name: color, dtype: int64

In [65]:
for i,val in enumerate(x['color']):
    if val == 'J':
        x['color'][i] = 50
    elif val == 'I':
        x['color'][i] = 100
    elif val == 'H':
        x['color'][i] = 150
    elif val == 'G':
        x['color'][i] = 200
    elif val == 'F':
        x['color'][i] = 250
    elif val == 'E':
        x['color'][i] = 300
    elif val == 'D':
        x['color'][i] = 350

ipykernel_launcher:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [66]:
df['clarity'].value_counts()

SI1     9751
VS2     9178
SI2     6828
VS1     6101
VVS2    3858
VVS1    2714
IF      1362
I1       553
Name: clarity, dtype: int64

In [67]:
for i, val in enumerate(x['clarity']):
    if val == 'I1':
        x['clarity'][i] = 50
    elif val == 'SI2':
        x['clarity'][i] = 100
    elif val == 'SI1':
        x['clarity'][i] = 150
    elif val == 'VS2':
        x['clarity'][i] = 200
    elif val == 'VS1':
        x['clarity'][i] = 250
    elif val == 'VVS2':
        x['clarity'][i] = 300
    elif val == 'VVS1':
        x['clarity'][i] = 350
    elif val == 'IF':
        x['clarity'][i] = 400

ipykernel_launcher:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [68]:
x

,carat,cut,color,clarity,depth,x,y,z
0,1.21,200,150,200,3.0,6.73,6.70,4.23
1,0.28,150,350,300,3.0,4.14,4.17,2.66
2,0.42,250,250,250,4.0,4.86,4.82,2.96
3,0.26,200,150,400,4.0,4.16,4.12,2.53
4,1.10,100,200,150,3.0,6.52,6.55,4.14
...,...,...,...,...,...,...,...,...
40340,1.55,250,150,200,4.0,7.46,7.39,4.55
40341,0.36,200,350,150,4.0,4.58,4.63,2.79
40342,0.57,150,100,200,4.0,5.33,5.34,3.32
40343,1.01,150,250,400,4.0,6.47,6.56,3.88


In [69]:
x[['cut', 'color', 'clarity']] = x[['cut', 'color', 'clarity']].astype(int)

In [70]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40345 entries, 0 to 40344
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    40345 non-null  float64
 1   cut      40345 non-null  int32  
 2   color    40345 non-null  int32  
 3   clarity  40345 non-null  int32  
 4   depth    40345 non-null  float64
 5   x        40345 non-null  float64
 6   y        40345 non-null  float64
 7   z        40345 non-null  float64
dtypes: float64(5), int32(3)
memory usage: 3.3 MB


In [71]:
scaler = StandardScaler()
standard_x = scaler.fit_transform(x)
standard_x

array([[ 0.88009271,  0.23281968, -0.83365837, ...,  0.89739454,
         0.84433944,  1.00174548],
       [-1.09526671, -0.7412662 ,  1.52591487, ..., -1.41784198,
        -1.3583998 , -1.26163392],
       [-0.79790077,  1.20690556,  0.34612825, ..., -0.77422411,
        -0.79247865, -0.82914104],
       ...,
       [-0.47929442, -0.7412662 , -1.42355167, ..., -0.35408466,
        -0.33974173, -0.31014958],
       [ 0.45528423, -0.7412662 ,  0.34612825, ...,  0.66497697,
         0.72244873,  0.49717046],
       [-0.54301569,  0.23281968,  0.93602156, ..., -0.35408466,
        -0.40068709, -0.48314673]])

In [72]:
x_train, x_test, y_train, y_test = train_test_split(standard_x, y, test_size=0.2, random_state= 42)

In [89]:
rand_forest = RandomForestRegressor( n_estimators= 1000)

In [90]:
rand_forest.fit(x_train, y_train)

RandomForestRegressor(n_estimators=1000)

In [91]:
rand_forest.score(x_test, y_test)

0.9799194543354257

In [92]:
predictions = rand_forest.predict(x_test)

In [93]:
np.sqrt(mean_squared_error(y_test, predictions))

564.8430748539827

In [78]:
rand_forest.fit(standard_x,y)

RandomForestRegressor(max_features=3, n_estimators=1000)

In [79]:
predictions = rand_forest.predict(x_test)

In [80]:
np.sqrt(mean_squared_error(y_test, predictions))

206.34914447427883

In [81]:
X_pred = pd.read_csv("diamonds_test.csv", index_col=0)
X_pred.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.30,Ideal,H,SI2,60.0,56.0,4.41,4.43,2.65
1,0.34,Ideal,D,IF,62.1,57.0,4.52,4.46,2.79
2,1.57,Very Good,I,VS2,60.3,58.0,7.58,7.55,4.56
3,0.31,Ideal,H,VS2,61.8,57.0,4.32,4.36,2.68
4,1.51,Good,I,VVS1,64.0,60.0,7.26,7.21,4.63


In [82]:
X_pred = X_pred.drop(['table'], axis=1)
X_pred

,carat,cut,color,clarity,depth,table,x,y,z
0,0.30,Ideal,H,SI2,60.0,56.0,4.41,4.43,2.65
1,0.34,Ideal,D,IF,62.1,57.0,4.52,4.46,2.79
2,1.57,Very Good,I,VS2,60.3,58.0,7.58,7.55,4.56
3,0.31,Ideal,H,VS2,61.8,57.0,4.32,4.36,2.68
4,1.51,Good,I,VVS1,64.0,60.0,7.26,7.21,4.63
...,...,...,...,...,...,...,...,...,...
13444,1.05,Ideal,F,SI2,60.3,55.0,6.55,6.68,3.99
13445,0.27,Very Good,G,VS1,61.9,56.0,4.13,4.16,2.56
13446,1.50,Premium,D,VS2,61.8,60.0,7.30,7.37,4.53
13447,1.70,Premium,F,SI2,60.2,61.0,7.79,7.68,4.66


In [83]:
for i, val in enumerate(X_pred['depth']):
    if val >= 59.5 and val <= 62.9:
        X_pred['depth'][i] = 4
    elif val > 56.2 and val < 59.5:
        X_pred['depth'][i] = 3
    elif val > 62.9 and val < 65:
        X_pred['depth'][i] = 3
    elif val >= 53.2 and val <= 56.2:
        X_pred['depth'][i] = 2
    elif val >= 65 and val <= 68:
        X_pred['depth'][i] = 2  

ipykernel_launcher:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [84]:
for i,val in enumerate(X_pred['cut']):
    if val == 'Fair':
        X_pred['cut'][i] = 50
    elif val == 'Good':
        X_pred['cut'][i] = 100
    elif val == 'Very Good':
       X_pred['cut'][i] = 150
    elif val == 'Ideal':
       X_pred['cut'][i] = 200
    elif val == 'Premium':
      X_pred['cut'][i] = 250

ipykernel_launcher:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [85]:
for i,val in enumerate(X_pred['color']):
    if val == 'J':
        X_pred['color'][i] = 50
    elif val == 'I':
        X_pred['color'][i] = 100
    elif val == 'H':
        X_pred['color'][i] = 150
    elif val == 'G':
        X_pred['color'][i] = 200
    elif val == 'F':
        X_pred['color'][i] = 250
    elif val == 'E':
        X_pred['color'][i] = 300
    elif val == 'D':
        X_pred['color'][i] = 350

ipykernel_launcher:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [86]:
for i, val in enumerate(X_pred['clarity']):
    if val == 'I1':
        X_pred['clarity'][i] = 50
    elif val == 'SI2':
        X_pred['clarity'][i] = 100
    elif val == 'SI1':
        X_pred['clarity'][i] = 150
    elif val == 'VS2':
        X_pred['clarity'][i] = 200
    elif val == 'VS1':
        X_pred['clarity'][i] = 250
    elif val == 'VVS2':
        X_pred['clarity'][i] = 300
    elif val == 'VVS1':
        X_pred['clarity'][i] = 350
    elif val == 'IF':
        X_pred['clarity'][i] = 400

ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [87]:
X_pred = scaler.fit_transform(X_pred)
X_pred

array([[-1.04815352,  0.23587598, -0.80575422, ..., -1.18640202,
        -1.17813028, -1.21874838],
       [-0.96499377,  0.23587598,  1.52194172, ..., -1.08875859,
        -1.15131317, -1.02904838],
       [ 1.59216846, -0.73901526, -1.3876782 , ...,  1.62750392,
         1.61084902,  1.36930158],
       ...,
       [ 1.4466389 ,  1.21076723,  1.52194172, ...,  1.37895702,
         1.44994637,  1.32865158],
       [ 1.86243764,  1.21076723,  0.35809375, ...,  1.81391409,
         1.72705649,  1.50480158],
       [-1.04815352,  0.23587598,  1.52194172, ..., -1.29292212,
        -1.2585816 , -1.23229838]])

In [88]:
predictions_submit = rand_forest.predict(X_pred)

ValueError: X has 9 features, but DecisionTreeRegressor is expecting 8 features as input.

In [48]:
predictions_submit

array([  468.42283333,  1516.9406    ,  9364.536     , ...,
       14026.764     , 10298.367     ,   788.398     ])

In [49]:
submission = pd.DataFrame({"id": range(len(predictions_submit)), "price": predictions_submit})

In [50]:
submission.shape

(13449, 2)

In [51]:
sample = pd.read_csv("sample_submission.csv")

In [52]:
sample.shape

(13449, 2)

In [53]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission_11.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [54]:
chequeator(submission)

You're ready to submit!
